### BeautifulSoup 사용해서 저스트와치 상세페이지 포스터 이미지 다운로드

In [7]:
# csv파일 읽어오기
import pandas as pd
csv_netflix1 = pd.read_excel("netflix_start_17년.xlsx")
csv_netflix1 .head()
csv_netflix1 = csv_netflix1.drop(["Unnamed: 0"], axis=1)
csv_netflix1 = csv_netflix1['start_2017']
csv_netflix1.loc[0]

'namsaegdaemun'

In [3]:
title_list =[]
openingdate_list = []
rate_list = []
genre_list = []
runtime_list = []
director_list = []
synopsis_list = []
ac_list = []

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib.request


def get_soup(target_url):
    html = urllib.request.urlopen(target_url)
    soup = BeautifulSoup(html, "html.parser")
    return soup

def extract_data(soup):
    try:
        detail_info = soup.find("div", class_="title-info visible-xs visible-sm")
    except IndexError:
        detail_info
    
    detile_infos = soup.find_all("div", class_="detail-infos")
    rate = detile_infos[0].text.strip().replace("평점","")
    genre = detile_infos[1].text.strip().replace("장르","")
    runtime = detile_infos[2].text.strip().replace("재생 시간","")
    director = detile_infos[3].text.strip().replace("감독","")
    actors = soup.find("div", class_="hidden-horizontal-scrollbar__items")
    actors = actors.find_all("a", class_="title-credit-name")
    actors_list = []
    for actor in actors:
        actor = actor.get_text().strip()
        actors_list.append(actor)         # 함수 내에서 for 문으로 뽑아낸 값을 actor_list로 저장
    actors_list = ",".join(actors_list)   #여기까지 tpye은 스트링 출력

    title = soup.find("div", class_="title-block")
    title_kor = title.find("h1").get_text().strip()
    opening_data = title.find("span", class_="text-muted").get_text().strip()    
    try:
        synopsis = soup.select("#base > div.jw-info-box > div > div.jw-info-box__container-content > div:nth-child(2) > div > div > div > p > span")[0].text.strip()
    except:
        synopsis=None
    
   
   
    title_list.append(title_kor)
    openingdate_list.append(opening_data)
    rate_list.append(rate)
    genre_list.append(genre)
    runtime_list.append(runtime)
    director_list.append(director)
    synopsis_list.append(synopsis)
    ac_list.append(actors_list)                 # 위 for 문에서 뽑아낸 데이터 정보를 함수 밖의 ac_list로 저장
    
    print("=======================================================================================================================================")
    print(title_kor, opening_data, rate, genre, runtime, director)
    print(synopsis)
    print(actors_list)    # 함수값 안에서의 출연진 값이 print 값으로 표시됨


for i in csv_netflix1[822:]:
    target_url = "https://www.justwatch.com/kr/%EC%98%81%ED%99%94/{}".format(i)
    soup = get_soup(target_url)
    extract_data(soup)

In [5]:
df = pd.DataFrame({
    "제목":title_list,'개봉일':openingdate_list,'평점':rate_list, '장르':genre_list, '재생시간':runtime_list, '감독':director_list,"출연진":ac_list, "시놉시스":synopsis_list})
df.tail(5)

,제목,개봉일,평점,장르,재생시간,감독,출연진,시놉시스
817,짱구는 못말려 극장판: 태풍을 부르는 장엄한 전설의 전쟁,(2002),75% 7.2,"애니메이션 , 역사",1시간 35분,Keiichi Hara,"Akiko Yajima,Miki Narahashi,Keiji Fujiwara,Sat...",어느 날 밤 짱구 가족은 동시에 사극에서 본 듯한 아름다운 여성이 나오는 꿈을 꾼다...
818,원더 휠,(2017),70% 6.2 (28k),"드라마 , 로맨스",1시간 41분,Woody Allen,"Jim Belushi,Juno Temple,Justin Timberlake,Kate...",웨이트리스로 일하는 지니는 해변의 안전요원 믹키와 사랑에 빠진다. 예상치 못한 캐...
819,베테랑,(2015),76% 7.0 (6k),"코미디 , 범죄 , 스릴러 , 액션",1시간 59분,Ryoo Seung-wan,"Hwang Jung-min,Yoo Ah-in,Yoo Hae-jin,Oh Dal-su...","한 번 꽂힌 것은 무조건 끝을 보는 행동파 서도철은 겁 없고, 못 잡는 것 없고, ..."
820,함께 있을 수 있다면,(2007),73% 6.8 (11k),"로맨스 , Made in Europe , 드라마",1시간 37분,Claude Berri,"Audrey Tautou,Guillaume Canet,Laurent Stocker,...",화가 지망생이지만 환경미화원으로 생계를 유지하고 있는 카미유. 적막한 도시 생활 속...
821,자객 섭은낭,(2015),70% 6.3 (16k),"드라마 , 액션 , 역사",1시간 45분,Hou Hsiao-hsien,"Shu Qi,Chang Chen,Nikki Hsieh,Sheu Fang-Yi,Eth...","9세기 초 당나라. 고위 관료의 딸로 태어났지만, 정혼자와의 이별 후 부패한 관리를..."


In [147]:
df.to_excel("test2019~20년.xlsx")

In [148]:
df.to_csv("test2019~20년.csv")